#**CIS 419/519**

## Logistic Regression worksheet

In this notebook, two examples for logistic regression are provided. In the first example, you will use the logistic regression module in `sklearn` to do a multi-class classification with the IRIS dataset. In the second example, you will explore using L1 regularization with logistic regression with the popular MNIST dataset. More information on the Logistic Regression function in `sklearn` can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression).

---
#Example 1 - Multi-class classification with logistic regression in `sklearn`

In this example, you will use the IRIS plant dataset to perform multi-class classification with the logistic regression function in the `sklearn` package. This example is adapted from [here](https://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html#sphx-glr-auto-examples-linear-model-plot-iris-logistic-py). For more information on the dataset can be found [here](https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-dataset).

---

As usual, the first step is to load the relevant libraries for computation and plotting.

In [ ]:
# Import numpy
import numpy as np

# Import libraries from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Import matplotlib for plotting
import matplotlib.pyplot as plt

Next, we load the IRIS dataset and create the logistic regression classifier. Then, we create the decision boundaries for visualization.

In [ ]:
# Import the dataset
iris    = datasets.load_iris()
X       = iris.data[:, :2]  # we only take the first two features.
y       = iris.target

#Use 20% of the data as test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create an instance of Logistic Regression Classifier and fit the data.

logreg  = LogisticRegression(penalty = 'none')
logreg.fit(X_train, y_train)



# Create the decision boundaries. We need to create a mesh and test the class of each point in the space.
# For that, we will assign a color to each point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max  = X_train[:, 0].min() - .5, X_train[:, 0].max() + .5
y_min, y_max  = X_train[:, 1].min() - .5, X_train[:, 1].max() + .5
h             = .02  # step size in the mesh
xx, yy        = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z             = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

Next, we plot the predictions results and training points in a color plot.

In [ ]:
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

---
#Example 2 - Exploring logistic regression with regularization

In this next example, we explore the implementation of L1 regularization with the logistic regression module in `sklearn`. This example is adapted from [here](https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html#sphx-glr-auto-examples-linear-model-plot-sparse-logistic-regression-mnist-py).

---
First, we load the required packages.

In [ ]:
# Import numpy
import numpy as np

# Import libraries from sklearn
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

# Import matplotlib for plotting
import matplotlib.pyplot as plt

Next, we load the MNIST dataset, perform pre-processing of the data using `StandardScaler`. `train_test_split` was used to partition training and test data.

Then, the logistic regression function `LogisticRegression` was called with the L1 regularization option, `penalty='l1'`.

Observe that with L1 regularization, we achieve sparsity in the number of coefficients required in the logistic regression model. In particular, 82.81% of the coefficients (`clf.coef_`) are zero.

In [ ]:
# Load data from https://www.openml.org/d/554
X, y          = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

In [ ]:
# Split the data into train and test
train_samples = 5000 # User-defined number of samples
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_samples, test_size=10000, random_state=0)

# Scale our data
scaler        = StandardScaler()
X_train       = scaler.fit_transform(X_train)
X_test        = scaler.transform(X_test)

# Turn up tolerance for faster convergence
# C is the inverse of our regularization parameter- the larger it is, the less regularization we have
clf           = LogisticRegression(C=100. / train_samples, penalty='l1', solver='saga', tol=0.1)
clf.fit(X_train, y_train)
sparsity      = np.mean(clf.coef_ == 0) * 100
score         = clf.score(X_test, y_test)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

Finally, we visualize our results with plots. Coefficients (`clf.coef_`) of each feature for each class (total 10 classes, digits 0-9) are plotted as images below (see `l1_plot.imshow`). In this MNIST example, the features corresponds directly to pixels in the images. Hence, by plotting these coefficients, we can identify (qualitatively) which parts of the images are the most critical in classifying the digits. As a side note, based on the color map used (`RdBu`), a darker blue shade represents a higher-than-average coefficient, while a darker red shade represents a lower-than-average coefficient.

**Question:** What do these plots tell you about a potential advantage of regression models over something like decision trees or knns?

In [ ]:
coef          = clf.coef_.copy()
plt.figure(figsize=(10, 5))
scale = np.abs(coef).max()
for i in range(10):
    l1_plot = plt.subplot(2, 5, i + 1)
    l1_plot.imshow(coef[i].reshape(28, 28), interpolation='nearest',
                   cmap=plt.cm.RdBu, vmin=-scale, vmax=scale)
    l1_plot.set_xticks(())
    l1_plot.set_yticks(())
    l1_plot.set_xlabel('Digit %i' % i)
plt.suptitle('Classification vectors for 10 classes:')

plt.show()